# RRAP-IS Provenance Demo Notebook

> A tutorial of RRAP authentication using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

## About

This notebook is a demonstration of generating a record of a model run and searching the provenance database to identify linkages between data, models, users and organisations with the RRAP-IS system.  

### Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [ ]:
import requests
import os
import sys
import json
import json2tree
from bs4 import BeautifulSoup
from IPython.display import IFrame, display, HTML, JSON, Markdown, Image
from mdsisclienttools.auth.TokenManager import DeviceFlowManager

import networkx as nx
import nx_altair as nxa
from networkx.readwrite import json_graph

import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
defaults = dict(width=800, height=600)
hv.opts.defaults(opts.EdgePaths(**defaults), opts.Graph(**defaults), opts.Nodes(**defaults))

import warnings
warnings.filterwarnings(action='once')

### Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [2]:
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display(f'Checking base urls')

for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed


## Authentication

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

[Return to Top](#toc)

In [ ]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

## Endpoint Documentation
Endpoint documentation can be found by appending either `/docs` or `/redoc` on the end a base URL.

For example:
<ul>
  <li><a href="https://prov-api.testing.rrap-is.com//redoc" target="_blank">Redoc FastAPI</a></li>
  <li><a href="https://prov-api.testing.rrap-is.com//docs" target="_blank">Docs FastAPI</a> </li>
</ul>

Then select from the menu an endpoint function call e.g. `/explore/lineage`

Then append the function call onto the base url e.g. `https://prov-api.testing.rrap-is.com/explore/lineage`

[Return to Top](#toc)

## Demonstration

### Register a model run
see <a href="https://prov-api.testing.rrap-is.com/redoc#tag/Model-Runs/operation/register_model_run_complete_model_run_register_complete_post" target="_blank">Endpoint documentation</a>

In [10]:
auth = token_manager.get_auth
postfix = "/model_run/register_complete"
payload = {
  "inputs": {
    "datasets": [
      {
        "id": "10378.1/1687286"
      }
    ],
    "config_files": [
      {
        "id": "10378.1/1687612"
      }
    ],
    "parameter_files": [
      {
        "id": "10378.1/1687695"
      }
    ]
  },
  "outputs": {
    "datasets": [
      {
        "id": "10378.1/1687280"
      }
    ]
  },
  "associations": {
    "model": {
      "id": "10378.1/1687693"
    },
    "modeller": {
      "id": "10378.1/1687696"
    },
    "requesting_organisation": {
      "id": "10378.1/1687697"
    }
  }
}
endpoint = prov_api + postfix 

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.post(endpoint, json=payload, auth=auth())


print(json.dumps(response.json()['status'], indent=2))

{
  "success": true,
  "details": "All successful."
}


### Explore lineage

In [6]:
auth = token_manager.get_auth
postfix = "/explore/lineage"
params = {
    "starting_id": "10378.1/1687588",
    "depth": 10
}
endpoint = prov_api + postfix

response = requests.get(endpoint, params=params, auth=auth())

result_graph = response.json()["graph"]

a_graph = json_graph.node_link_graph(result_graph)
im = hv.Graph.from_networkx(a_graph, nx.layout.fruchterman_reingold_layout)#.opts(tools=['hover'])

hv.save(im, 'lineage_network.html', backend='bokeh')

HTML('lineage_network.html')

### Explore Children

In [7]:
auth = token_manager.get_auth
postfix = "/explore/children"
params = {
    "starting_id": "10378.1/1687588"
}
endpoint = prov_api + postfix 

response = requests.get(endpoint, params=params, auth=auth())

result_graph = response.json()["graph"]

a_graph = json_graph.node_link_graph(result_graph)
im = hv.Graph.from_networkx(a_graph, nx.layout.fruchterman_reingold_layout).opts(tools=['hover'])

hv.save(im, 'network.html', backend='bokeh')
HTML('network.html')

[Return to Top](#toc)